- environment setup

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score

In [3]:
import wrangle

from wrangle import clean_311, split_separate_scale

In [27]:
def keep_info(df):
    df.drop(df.columns.difference(['dept','call_reason', 'source_id',
                                   'council_district', 'resolution_days_due']), 1, inplace=True)
    return df

SyntaxError: invalid syntax (<ipython-input-27-b0b2b5d2fa07>, line 1)

In [4]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

- Wrangle 

In [5]:
# acquire data
df = pd.read_csv('allservicecalls.csv')

In [6]:
df.head()

,Category,CASEID,OPENEDDATETIME,SLA_Date,CLOSEDDATETIME,Late (Yes/No),Dept,REASONNAME,TYPENAME,CaseStatus,SourceID,OBJECTDESC,Council District,XCOORD,YCOORD,Report Starting Date,Report Ending Date
0,Graffiti,1010444245,2012-08-15T00:00:00,2012-08-30T00:00:00,NaN,YES,Code Enforcement Services,Graffiti,Graffiti Public Property,Open,Web Portal,"600 NOGALITOS ST, San Antonio, 78204",5,2125683.0,13695548.0,2020-05-15T00:00:00,2021-05-15T00:00:00
1,Property Maintenance,1010888252,2013-06-06T00:00:00,2013-08-09T00:00:00,NaN,YES,Code Enforcement Services,Code Enforcement (IntExp),Alley-Way Maintenance,Open,Web Portal,"6043 CASTLE QUEEN, San Antonio, 78218",2,2169702.0,13725769.0,2020-05-15T00:00:00,2021-05-15T00:00:00
2,Property Maintenance,1010966128,2013-07-19T00:00:00,2013-09-23T00:00:00,NaN,YES,Code Enforcement Services,Code Enforcement (IntExp),Junk Vehicle On Private Property,Open,Web Portal,"842 KIRK PL, San Antonio, 78226",5,2116192.0,13692260.0,2020-05-15T00:00:00,2021-05-15T00:00:00
3,Property Maintenance,1011052825,2013-09-16T00:00:00,2013-09-30T00:00:00,NaN,YES,Code Enforcement Services,Code Enforcement,Right Of Way/Sidewalk Obstruction,Open,Internal Services Requests,"54 KENROCK RIDGE, San Antonio, 78254",7,2082242.0,13737817.0,2020-05-15T00:00:00,2021-05-15T00:00:00
4,Property Maintenance,1011052826,2013-09-16T00:00:00,2013-09-30T00:00:00,NaN,YES,Code Enforcement Services,Code Enforcement,Right Of Way/Sidewalk Obstruction,Open,Internal Services Requests,"74 KENROCK RIDGE, San Antonio, 78254",7,2082389.0,13737877.0,2020-05-15T00:00:00,2021-05-15T00:00:00


In [7]:
df.shape

(495440, 17)

In [8]:
# clean data
df = clean_311(df)

In [9]:
df.head()

,case_id,open_date,due_date,closed_date,is_late,dept,call_reason,case_type,case_status,source_id,address,council_district,longitude,latitude,days_open,resolution_days_due,days_before_or_after_due,level_of_delay,district_0,district_1,district_2,district_3,district_4,district_5,district_6,district_7,district_8,district_9,district_10,zipcode
2497,1014281655,2018-02-28,2018-07-09,2020-06-06,YES,Code Enforcement Services,code,Certificates of Occupancy,Closed,Internal Services Requests,"5550 EASTERLING, San Antonio, 78251",6,2067118.0,13723868.0,829.0,131,-698.0,Extremely Late Response,0,0,0,0,0,0,1,0,0,0,0,78251
2498,1014281665,2018-02-28,2018-07-09,2020-06-06,YES,Code Enforcement Services,code,Zoning: District Requirement,Closed,Internal Services Requests,"5550 EASTERLING, San Antonio, 78251",6,2067118.0,13723868.0,829.0,131,-698.0,Extremely Late Response,0,0,0,0,0,0,1,0,0,0,0,78251
2499,1014281677,2018-02-28,2018-07-09,2020-06-06,YES,Code Enforcement Services,code,Certificates of Occupancy,Closed,Internal Services Requests,"5550 EASTERLING, San Antonio, 78251",6,2067118.0,13723868.0,829.0,131,-698.0,Extremely Late Response,0,0,0,0,0,0,1,0,0,0,0,78251
2556,1014300321,2018-03-06,2020-09-24,2020-05-18,NO,Development Services,buildings,Street Light Existing Res Sub Div,Closed,Constituent Call,"1018 KING AVE, San Antonio, 78211",4,2108015.0,13680344.0,804.0,933,129.0,Very Early Response,0,0,0,0,1,0,0,0,0,0,0,78211
2657,1014326068,2018-03-14,2018-07-23,2020-05-27,YES,Code Enforcement Services,code,Zoning: District Requirement,Closed,Internal Services Requests,"11800 FISCHER RD, San Antonio, 78073",4,2091672.0,13658660.0,805.0,131,-674.0,Extremely Late Response,0,0,0,0,1,0,0,0,0,0,0,78073


In [10]:
df.shape

(428870, 30)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 428870 entries, 2497 to 495432
Data columns (total 30 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   case_id                   428870 non-null  int64         
 1   open_date                 428870 non-null  datetime64[ns]
 2   due_date                  428870 non-null  datetime64[ns]
 3   closed_date               428870 non-null  datetime64[ns]
 4   is_late                   428870 non-null  object        
 5   dept                      428870 non-null  object        
 6   call_reason               428870 non-null  object        
 7   case_type                 428870 non-null  object        
 8   case_status               428870 non-null  object        
 9   source_id                 428870 non-null  object        
 10  address                   428870 non-null  object        
 11  council_district          428870 non-null  int64         
 12 

In [12]:
train, validate, test, X_train, y_train, X_validate, y_validate, X_test, y_test, train_scaled, validate_scaled, test_scaled = split_separate_scale(df)

In [13]:
train.head()

,case_id,open_date,due_date,closed_date,is_late,dept,call_reason,case_type,case_status,source_id,address,council_district,longitude,latitude,days_open,resolution_days_due,days_before_or_after_due,level_of_delay,district_0,district_1,district_2,district_3,district_4,district_5,district_6,district_7,district_8,district_9,district_10,zipcode
260102,1016484102,2020-10-30,2020-11-04,2020-10-31,NO,Solid Waste Management,waste,No Pickup,Closed,Web Portal,"1302 W HARDING BLVD, SAN ANTONIO, 78221",3,2122015.0,13675610.0,1.0,5,4.0,Early Response,0,0,0,1,0,0,0,0,0,0,0,78221
202138,1016390561,2020-09-10,2020-09-17,2020-09-14,NO,Solid Waste Management,customer_service,Customer Service(Complaint),Closed,Web Portal,"3418 LONE VALLEY, SAN ANTONIO, 78247",10,2149891.0,13757711.0,4.0,7,3.0,Early Response,0,0,0,0,0,0,0,0,0,0,1,78247
210324,1016401694,2020-09-17,2020-09-19,2020-09-17,NO,Solid Waste Management,misc,Dead Animal Pick Up,Closed,Web Portal,"1903 SAUNDERS, SAN ANTONIO, 78207",5,2121436.0,13702209.0,0.0,2,2.0,Early Response,0,0,0,0,0,1,0,0,0,0,0,78207
182502,1016364931,2020-08-25,2020-10-28,2020-08-31,NO,Development Services,code,Overgrown Yard/Trash,Closed,Web Portal,"615 CLEARCREST, SAN ANTONIO, 78227",6,2081335.0,13704368.0,6.0,64,58.0,Early Response,0,0,0,0,0,0,1,0,0,0,0,78227
219960,1016416005,2020-09-25,2020-09-29,2020-09-25,NO,Unknown,traffic,Traffic Signals (Maintenance_Emergency),Closed,Web Portal,BANDERA RD and MAINLAND DR,7,2084733.0,13735038.0,0.0,4,4.0,Early Response,0,0,0,0,0,0,0,1,0,0,0,NaN


In [14]:
train.shape

(240167, 30)

In [15]:
train_scaled.shape

(240167, 18)

In [16]:
y_train.shape

(240167,)

# Establish Baseline

In [17]:
# look at values of target variable

y_train.value_counts()

Early Response              181108
On Time Response             33203
Very Early Response          17279
Late Response                 5839
Very Late Response            2021
Extremely Late Response        635
Extremely Early Response        82
Name: level_of_delay, dtype: int64

In [18]:
# set up as dataframes
y_train = pd.DataFrame(dict(actual=y_train))
y_validate = pd.DataFrame(dict(actual=y_validate))
y_test = pd.DataFrame(dict(actual=y_test))

In [19]:
# 'Early Response' is by far the most frequent value so that will be our baseline
y_train['baseline'] = 'Early Response'

In [20]:
y_train.head()

,actual,baseline
260102,Early Response,Early Response
202138,Early Response,Early Response
210324,Early Response,Early Response
182502,Early Response,Early Response
219960,Early Response,Early Response


In [21]:
# calculate accuracy of baseline

print(' Baseline Accuracy: {:.2%}'.format(accuracy_score(y_train.actual, y_train.baseline)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(y_train.baseline, y_train.actual))
print('---')
print(classification_report(y_train.actual, y_train.baseline))

 Baseline Accuracy: 75.41%
---
Confusion Matrix
actual          Extremely Late Response  Very Late Response  Late Response  \
baseline                                                                     
Early Response                      635                2021           5839   

actual          On Time Response  Early Response  Very Early Response  \
baseline                                                                
Early Response             33203          181108                17279   

actual          Extremely Early Response  
baseline                                  
Early Response                        82  
---
                          precision    recall  f1-score   support

          Early Response       0.75      1.00      0.86    181108
Extremely Early Response       0.00      0.00      0.00        82
 Extremely Late Response       0.00      0.00      0.00       635
           Late Response       0.00      0.00      0.00      5839
        On Time Response       0.

#### Takeaways

- Baseline of 'Early Response' is correct about 75% of the time

- This could be a difficult baseline to beat 

# Modeling

In [22]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 240167 entries, 260102 to 33841
Data columns (total 30 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   case_id                   240167 non-null  int64         
 1   open_date                 240167 non-null  datetime64[ns]
 2   due_date                  240167 non-null  datetime64[ns]
 3   closed_date               240167 non-null  datetime64[ns]
 4   is_late                   240167 non-null  object        
 5   dept                      240167 non-null  object        
 6   call_reason               240167 non-null  object        
 7   case_type                 240167 non-null  object        
 8   case_status               240167 non-null  object        
 9   source_id                 240167 non-null  object        
 10  address                   240167 non-null  object        
 11  council_district          240167 non-null  int64         
 12

In [23]:
# drop columns that contain the answer to prevent answer leaking

train = train.drop(columns = ['closed_date', 'is_late', 'days_before_or_after_due', 'days_open'])
validate = validate.drop(columns = ['closed_date', 'is_late', 'days_before_or_after_due', 'days_open'])
test = test.drop(columns = ['closed_date', 'is_late', 'days_before_or_after_due', 'days_open'])

X_train = X_train.drop(columns = ['closed_date', 'is_late', 'days_before_or_after_due', 'days_open'])
X_validate = X_validate.drop(columns = ['closed_date', 'is_late', 'days_before_or_after_due', 'days_open'])
X_test = X_test.drop(columns = ['closed_date', 'is_late', 'days_before_or_after_due', 'days_open'])

train_scaled = train_scaled.drop(columns = ['days_before_or_after_due', 'days_open'])
validate_scaled = validate_scaled.drop(columns = ['days_before_or_after_due', 'days_open'])
test_scaled = test_scaled.drop(columns = ['days_before_or_after_due', 'days_open'])

### Model 1: Logistic Regression - Default Parameters / All Features

In [24]:
# make logistic regression object
log1 = LogisticRegression(random_state=123)